In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config.from_object('otsukare.config.BaseConfig')

db = SQLAlchemy(app)

C:\Users\Damnt\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:839: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
C:\Users\Damnt\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:839: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [2]:
from otsukare.models import *

In [ ]:
session.query(
    Expense.date,
    func.sum(Expense.value).label('total')
).join(Expense.cost_center
).filter(CostCenter.id.in_([2, 3])
).group_by(Expense.date
).all()

In [14]:
from otsukare.analysis import * 

In [31]:
kana_known = pd.read_sql('SELECT * from "Kana_Known"', db.engine.connect().connection)[["user_id","tokens"]].groupby("user_id").sum()
kana_known.columns = ["Kana"]
words_known = pd.read_sql('SELECT * from "Words_Known"', db.engine.connect().connection)[["user_id","tokens"]].groupby("user_id").sum()
words_known.columns = ["Words"]
sent_known = pd.read_sql('SELECT * from "Needs_Known"', db.engine.connect().connection)[["user_id","tokens"]].groupby("user_id").sum()
sent_known.columns = ["Sentences"]

usernames = pd.read_sql('SELECT * from "Users"', db.engine.connect().connection).set_index("id")[["username"]]
usernames.columns = ["Username"]

usernames.join(kana_known).join(words_known).join(sent_known).fillna(0)

,Username,Kana,Words,Sentences
id,,,,
1,Bluemania,23.0,2.0,2.0
2,Skyver,0.0,0.0,0.0


In [3]:
import pandas as pd

In [4]:
kana_known = pd.read_sql(Kana_Known.query.filter_by(user_id=1).statement, Kana_Known.query.session.bind)

In [5]:
kana = pd.read_sql(Kana.query.statement, Kana.query.session.bind)

In [6]:
kana_stats = pd.merge(kana_known, kana, how='inner', left_on="kana_id", right_on="id")

In [7]:
kana_stats = kana_stats.set_index("kana_id")

In [8]:
task_stats = pd.read_sql(Tasks.query.filter(Tasks.user_id==1, 
                                      Tasks.task_id>=1, Tasks.task_id<=5, 
                                      Tasks.status!=-1).statement, Kana.query.session.bind)

In [9]:
incorrect = task_stats[task_stats["status"]==0].groupby("answer").count()["id"]
incorrect.index = incorrect.index.astype(int)
incorrect.name = "incorrect"
correct = task_stats[task_stats["status"]==1].groupby("answer").count()["id"]
correct.index = correct.index.astype(int)
correct.name = "correct"

In [10]:
correct

answer
101    2
102    1
105    1
110    2
114    2
142    2
160    1
162    2
163    3
168    2
179    2
185    2
196    2
207    2
208    2
21     1
27     3
37     2
38     2
40     2
50     2
6      1
66     2
85     1
90     3
91     1
99     2
Name: correct, dtype: int64

In [11]:
kana_stats = kana_stats.join(correct).join(incorrect)

In [12]:
kana_stats.columns

Index(['id_x', 'user_id', 'level', 'last_practiced', 'tokens', 'id_y', 'type',
       'kana', 'romanji', 'tags', 'correct', 'incorrect'],
      dtype='object')

In [13]:
kana_stats = kana_stats[["kana", "romanji", "correct", "incorrect"]]

In [14]:
kana_stats

,kana,romanji,correct,incorrect
kana_id,,,,
144,ジュ,ju,NaN,4.0
91,みゃ,mya,1.0,1.0
160,ミュ,myu,1.0,1.0
66,よ,yo,2.0,NaN
37,ね,ne,2.0,1.0
50,ろ,ro,2.0,NaN
110,ボ,bo,2.0,1.0
207,ゼ,ze,2.0,1.0
38,に,ni,2.0,NaN


In [15]:
test = task_stats[(task_stats["status"]==0) & (task_stats["response"]!="-1")]

In [16]:
most_incorrect_x = pd.crosstab(test["answer"],test["response"])
most_incorrect = most_incorrect_x.idxmax(axis=1)


In [17]:
most_incorrect.index = most_incorrect.index.astype(int)


In [18]:
most_incorrect = kana_stats.loc[most_incorrect.index,"kana"]
most_incorrect.name = "most incorrect"

In [19]:
kana_stats = kana_stats.join(most_incorrect)

In [20]:
kana_stats

,kana,romanji,correct,incorrect,most incorrect
kana_id,,,,,
144,ジュ,ju,NaN,4.0,ジュ
91,みゃ,mya,1.0,1.0,みゃ
160,ミュ,myu,1.0,1.0,ミュ
66,よ,yo,2.0,NaN,NaN
37,ね,ne,2.0,1.0,ね
50,ろ,ro,2.0,NaN,NaN
110,ボ,bo,2.0,1.0,ボ
207,ゼ,ze,2.0,1.0,ゼ
38,に,ni,2.0,NaN,NaN
